In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [9]:
from medvqa.datasets.iuxray import IUXRAY_REPORTS_MIN_JSON_PATH
from medvqa.utils.files import (
    get_checkpoint_folder_path,
    get_results_folder_path,
    load_json, load_pickle,
    save_pickle,
)
from nltk import sent_tokenize

def _load_iuxray_sentences():
    print(f'Loading iuxray reports from {IUXRAY_REPORTS_MIN_JSON_PATH}')
    reports = load_json(IUXRAY_REPORTS_MIN_JSON_PATH)
    sentences = set()
    for r in reports.values():
        findings = r['findings']
        impression = r['impression']
        for x in (findings, impression):
            if x:
                for s in sent_tokenize(x):
                    s = ' '.join(s.split()) # Remove extra spaces
                    sentences.add(s)
    sentences = list(sentences)
    sentences.sort(key=lambda x: (len(x), x)) # Sort by length and then alphabetically
    sentences = [s for s in sentences if any(c.isalpha() for c in s)] # Remove sentences without any alphabetic character
    print(f'Number of sentences: {len(sentences)}')
    print(f'Shortest sentence: {sentences[0]}')
    print(f'Longest sentence: {sentences[-1]}')
    return sentences

sentences = _load_iuxray_sentences()

print('Loading CheXpert labeler')
from medvqa.metrics.medical.chexpert import ChexpertLabeler
chexpert_labeler = ChexpertLabeler()
chexpert_labels = chexpert_labeler.get_labels(sentences, update_cache_on_disk=True)

print('Loading CheXbert labeler')
from medvqa.metrics.medical.chexbert import CheXbertLabeler
chexbert_labeler = CheXbertLabeler()
chexbert_labels = chexbert_labeler.get_labels(sentences, update_cache_on_disk=True)

print('Loading RadGraph labeler')
from medvqa.metrics.medical.radgraph import RadGraphLabeler
radgraph_labeler = RadGraphLabeler()
radgraph_labels = radgraph_labeler.get_labels(sentences, update_cache_on_disk=True)    

Loading iuxray reports from /mnt/workspace/iu-x-ray/dataset/reports/reports.min.json
Number of sentences: 8617
Shortest sentence: KUB.
Longest sentence: The infrahilar pulmonary markings appear slightly prominent bilaterally, which XXXX represents XXXX appearance for the patient but difficult to completely exclude some reactive airway/bronchitic changes in the absence of comparison radiographs.. No airspace consolidation or lobar atelectasis.
Loading CheXpert labeler
Loading CheXbert labeler


8617it [00:00, 18598.40it/s]


Loading RadGraph labeler


In [44]:
import numpy as np
from medvqa.utils.metrics import jaccard_between_dicts

i = 3001

relevant_sentences = []
if len(radgraph_labels[i]) > 0:
    for j in range(len(sentences)):
        if i != j:
            js = jaccard_between_dicts(radgraph_labels[i], radgraph_labels[j])
            if js >= 0.4 or ((np.all(chexpert_labels[i] == chexpert_labels[j]) or \
                np.all(chexbert_labels[i] == chexbert_labels[j])) and js >= 0.2):
                relevant_sentences.append(j)
len(relevant_sentences)

221

In [45]:
print(sentences[i])
print()
for j in range(min(40, len(relevant_sentences))):
    print(sentences[relevant_sentences[j]])

Minimal degenerative disease thoracic spine

Thoracic spine.
Degenerative spine.
DISH of the thoracic spine.
Degenerative changes spine.
Minimal thoracic spondylosis.
Degenerative changes the spine.
Degenerative change of the spine.
Degenerative changes in the spine.
Degenerative changes of the spine.
Scattered thoracic spine spurring.
Degenerative changes of the spine..
Degenerative changes thoracic spine.
Levoscoliosis of the thoracic spine.
Degenerative changes of the the spine.
Dextroscoliosis of the thoracic spine.
degenerative changes within the spine.
Degenerative changes are present spine.
Degenerative changes of thoracic spine.
Degenerative disease of thoracic spine.
Mild degenerative changes in the spine.
Mild degenerative changes of the spine.
Mild thoracic spine degenerative change
Degenerative changes noted in the spine.
Degenerative spurring of thoracic spine.
Mild degenerative change thoracic spine.
Mild thoracic spine degenerative change.
No XXXX fractures of the thorac

In [70]:
chexpert_labels.shape, chexbert_labels.shape

((8617, 14), (8617, 14))

In [72]:
radgraph_labels[1]

{(11, 652562049788717786): 1}

In [6]:
from medvqa.eval_fact_embedding_on_ranking import _load_mimiccxr_radiologist_annotations

In [7]:
sentences, labels = _load_mimiccxr_radiologist_annotations()

In [11]:
import numpy as np

In [37]:
def score(a, b):
    for x, y in zip(a, b):
        if x == -2:
            continue
        if x != y:
            return 0
    return 1

In [41]:
sorted_idxs[0]

0

In [42]:
labels[0]

array([ 1, -2, -2, -2, -2, -2, -2,  1, -2, -2, -2, -2, -2, -2], dtype=int8)

In [66]:
idx = 11
print(sentences[idx])
sorted_idxs = np.argsort([score(labels[idx], x) or score(x, labels[idx]) for x in labels])[::-1]
print()
for i in range(100):
    j = sorted_idxs[i]
    print(sentences[j],  np.mean(labels[j] == labels[idx]), score(labels[idx], labels[j]) or score(labels[j], labels[idx]))

recurrent right lower lobe patchy airspace opacity , suggestive of waxing and waning atelectasis .

band of atelectasis crossing the left hilus has been present for several days . 0.8571428571428571 1
lungs are clear except for minimal patchy atelectasis in the right infrahilar region . 0.8571428571428571 1
lungs are clear except for minimal lingular atelectasis or scar . 0.8571428571428571 1
left lung is clear except for minor linear atelectasis in the left mid and lower lung regions . 0.8571428571428571 1
lungs are clear except for linear atelectasis at the lung bases , left greater than right . 0.8571428571428571 1
aside from mild atelectasis at the right base , lungs are clear . 0.8571428571428571 1
aside from atelectasis in the right base the lungs are clear . 0.8571428571428571 1
aside from atelectasis , left lung is grossly clear . 0.8571428571428571 1
no substantial atelectasis except for the retrocardiac lung area . 0.8571428571428571 1
left lung is clear aside from minimal le

In [18]:
from importlib import reload
import medvqa

In [24]:
reload(medvqa.eval_fact_embedding_on_ranking)

<module 'medvqa.eval_fact_embedding_on_ranking' from '/home/pamessina/medvqa/medvqa/eval_fact_embedding_on_ranking.py'>

In [25]:
from medvqa.eval_fact_embedding_on_ranking import SentenceRanker

In [26]:
sr = SentenceRanker('mimiccxr_rad_annotations')

In [63]:
query_idx = 11
output = sr.rank_sentences(query_idx, model_name='microsoft/BiomedVLP-CXR-BERT-specialized',
                           checkpoint_folder_path='/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_152825_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)',
                           top_k=100)

100%|███████████████████████████████████████████████████████████████████████████| 1165/1165 [00:00<00:00, 401905.26it/s]


In [56]:
query_idx = 6
output = sr.rank_sentences(query_idx, model_name='emilyalsentzer/Bio_ClinicalBERT', average_token_embeddings=True,
                           top_k=100)

100%|███████████████████████████████████████████████████████████████████████████| 1165/1165 [00:00<00:00, 434111.95it/s]


In [64]:
print(sr.sentences[query_idx])
print()
output

recurrent right lower lobe patchy airspace opacity , suggestive of waxing and waning atelectasis .



(['recurrent right lower lobe patchy airspace opacity , suggestive of waxing and waning atelectasis .',
  'pneumonia or atelectasis at the right lung base .',
  'there is patchy opacity at the right base which could reflect compressive atelectasis .',
  'right lower lobe collapse is persistent .',
  'consolidation continues to progress in the right lower lobe .',
  'patchy opacity at the right base may represent partial lower lobe atelectasis , although pneumonia or aspiration cannot be entirely excluded .',
  'subtle opacification at the right lung base , which may represent atelectasis , however an early developing pneumonia is a consideration .',
  'subtle opacity at the base of the right lung could represent atelectasis however infection should be considered in the appropriate clinical setting .',
  'focal opacity at the right lung base has increased , either an atelectasis or consolidation .',
  'subtle patchy right base opacity could be due to aspiration , infection or atelectasi

# MIMIC-CXR Radiologist Annotations

#### microsoft/BiomedVLP-CXR-BERT-specialized

In [216]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=116,1396652957813721259).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 128)
100%|█████████████████████████████████████| 1165/1165 [00:00<00:00, 5315.63it/s]
Embeddings shape: (1165, 128)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as que

#### microsoft/BiomedVLP-BioViL-T

In [217]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-BioViL-T" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-BioViL-T
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=104,2678825193822464594).pkl
len(self.cache["hashes"]) = 4451804
self.cache["embeddings"].shape = (4451804, 128)
100%|██████████████████████████████████████| 1165/1165 [00:04<00:00, 236.70it/s]
Embeddings shape: (1165, 128)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as query
100%|██

#### michiyasunaga/BioLinkBERT-large (cls)

In [218]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "michiyasunaga/BioLinkBERT-large" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: michiyasunaga/BioLinkBERT-large
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=107,40983794350539675).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 1024)
100%|█████████████████████████████████████| 1165/1165 [00:00<00:00, 1517.76it/s]
Embeddings shape: (1165, 1024)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as query
100%|█

#### michiyasunaga/BioLinkBERT-large (tok)

In [219]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "michiyasunaga/BioLinkBERT-large" \
--batch_size 100 \
--average_token_embeddings

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: michiyasunaga/BioLinkBERT-large
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: True
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=132,1253761809285128054).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 1024)
100%|█████████████████████████████████████| 1165/1165 [00:00<00:00, 1489.36it/s]
Embeddings shape: (1165, 1024)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as query
100%|

#### microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext (cls)

In [220]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=137,1948119357925567498).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 768)
100%|█████████████████████████████████████| 1165/1165 [00:00<00:00, 1819.72it/s]
Embeddings shape: (1165, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with

#### microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext (cls)

In [221]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext" \
--batch_size 100 \
--average_token_embeddings

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: True
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=162,1758876944325999998).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 768)
100%|█████████████████████████████████████| 1165/1165 [00:00<00:00, 1819.70it/s]
Embeddings shape: (1165, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with 

#### emilyalsentzer/Bio_ClinicalBERT (cls)

In [222]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "emilyalsentzer/Bio_ClinicalBERT" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: emilyalsentzer/Bio_ClinicalBERT
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=107,1542209846559636901).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 768)
100%|█████████████████████████████████████| 1165/1165 [00:00<00:00, 1776.79it/s]
Embeddings shape: (1165, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as query
100%|█

#### emilyalsentzer/Bio_ClinicalBERT (tok)

In [223]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "emilyalsentzer/Bio_ClinicalBERT" \
--batch_size 100 \
--average_token_embeddings

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: emilyalsentzer/Bio_ClinicalBERT
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: True
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=132,412155249657523952).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 768)
100%|█████████████████████████████████████| 1165/1165 [00:00<00:00, 3205.03it/s]
Embeddings shape: (1165, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as query
100%|███

#### CheXbert

In [235]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "CheXbert" \
--device "cuda" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: CheXbert
   device: cuda
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
100%|███████████████████████████████████████| 1165/1165 [00:13<00:00, 85.50it/s]
Embeddings shape: (1165, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as query
100%|█████████████████████████████████████| 1165/1165 [00:01<00:00, 1125.10it/s]
mean_AUC: 0.8698
mean_relevant: 57.8080
count: 1156 / 1165 (99.23%)


#### CXR Fact Encoder (T)

In [224]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230927_194923_MIMIC-CXR(triplets-only)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230927_194923_MIMIC-CXR(triplets-only)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
checkpoint_names = ['checkpoint_40_ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9355.pt']
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=359,388447562913600091).pkl
len(self.cache["hashes"]) = 144285
self.ca

#### CXR Fact Encoder (T -> T+EC)

In [227]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231003_001641_MIMIC-CXR(triplets+entcont)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231003_001641_MIMIC-CXR(triplets+entcont)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
checkpoint_names = ['checkpoint_1_encc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9444.pt']
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=366,4246634492375333681).pkl
len(self.cache["hashes"]) = 13485

#### CXR Fact Encoder (T -> T+C)

In [228]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_013825_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_013825_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
checkpoint_names = ['checkpoint_64_cacc+chf1+chf1+cscc+hscc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9474.pt']
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=387,2492157991032224976).pkl
len(self.cac

In [4]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_155915_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_155915_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
checkpoint_names = ['checkpoint_29_cacc+chf1+chf1+cscc+hscc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9495.pt']
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=387,1308360022390228203).pkl
len(self.cach

#### CXR Fact Encoder (T -> T+EC+NLI)

In [229]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_220207_MIMIC-CXR(triplets+entcon+nli)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_220207_MIMIC-CXR(triplets+entcon+nli)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
checkpoint_names = ['checkpoint_52_encc+nlcc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9002.pt']
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=375,1493189425621868816).pkl
len(self.cache["hashes"]

#### CXR Fact Encoder (T -> T+R)

In [230]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_110825_MIMIC-CXR(triplets+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_110825_MIMIC-CXR(triplets+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
checkpoint_names = ['checkpoint_93_spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9415.pt']
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=368,1866338409828607810).pkl
len(self.cache["hashes"]) = 134

#### CXR Fact Encoder (T -> T+C+EC+NLI+R)

In [231]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_152825_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_152825_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
checkpoint_names = ['checkpoint_26_cacc+chf1+chf1+cscc+encc+hscc+nlcc+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9386.pt']
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=423,1

In [3]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "mimiccxr_radiologist_annotations" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_174626_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/" \
--batch_size 100

script's arguments:
   evaluation_mode: mimiccxr_radiologist_annotations
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_174626_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/
   distance_metric: cosine
   average_token_embeddings: False
Loading mimiccxr sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr_sentences_and_relevant.pkl
len(sentences): 1165
len(relevant_sentences): 1165
checkpoint_names = ['checkpoint_50_cacc+chf1+chf1+cscc+encc+hscc+nlcc+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9373.pt']
  0%|                                                  | 0/1165 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=423,3

# IU X-ray Automatic Labelers (chexpert + chexbert + radgraph)

#### microsoft/BiomedVLP-CXR-BERT-specialized

In [47]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray reports from /mnt/workspace/iu-x-ray/dataset/reports/reports.min.json
Number of sentences: 8617
Shortest sentence: KUB.
Longest sentence: The infrahilar pulmonary markings appear slightly prominent bilaterally, which XXXX represents XXXX appearance for the patient but difficult to completely exclude some reactive airway/bronchitic changes in the absence of comparison radiographs.. No airspace consolidation or lobar atelectasis.
Loading CheXpert labeler
Loading CheXbert labeler
8617it [00:00, 17841.73it/s]
Loading RadGraph labeler
100%|███████████████████████████████████████| 8617/8617 [03:07<00:00, 45.92it/s]
Saving iuxray sentences and relevant sentences to /mnt/data/pamessina/workspaces/medvqa-w

#### microsoft/BiomedVLP-BioViL-T

In [48]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-BioViL-T" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-BioViL-T
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=104,2678825193822464594).pkl
len(self.cache["hashes"]) = 4451804
self.cache["embeddings"].shape = (4451804, 128)
100%|█████████████████████████████████████| 8617/8617 [00:03<00:00, 2565.91it/s]
Embeddings shape: (8617, 128)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as 

#### michiyasunaga/BioLinkBERT-large (cls)

In [49]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "michiyasunaga/BioLinkBERT-large" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: michiyasunaga/BioLinkBERT-large
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=107,40983794350539675).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 1024)
100%|████████████████████████████████████| 8617/8617 [00:00<00:00, 26704.83it/s]
Embeddings shape: (8617, 1024)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as

#### michiyasunaga/BioLinkBERT-large (tok)

In [50]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "michiyasunaga/BioLinkBERT-large" \
--batch_size 100 \
--average_token_embeddings

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: michiyasunaga/BioLinkBERT-large
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: True
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=132,1253761809285128054).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 1024)
100%|████████████████████████████████████| 8617/8617 [00:00<00:00, 26889.74it/s]
Embeddings shape: (8617, 1024)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence a

#### microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext (cls)

In [51]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=137,1948119357925567498).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 768)
100%|████████████████████████████████████| 8617/8617 [00:00<00:00, 32819.15it/s]
Embeddings shape: (8617, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on rank

#### microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext (tok)

In [52]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext" \
--batch_size 100 \
--average_token_embeddings

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: True
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=162,1758876944325999998).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 768)
100%|████████████████████████████████████| 8617/8617 [00:00<00:00, 33578.34it/s]
Embeddings shape: (8617, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranki

#### emilyalsentzer/Bio_ClinicalBERT (cls)

In [53]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "emilyalsentzer/Bio_ClinicalBERT" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: emilyalsentzer/Bio_ClinicalBERT
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=107,1542209846559636901).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 768)
100%|████████████████████████████████████| 8617/8617 [00:00<00:00, 33897.08it/s]
Embeddings shape: (8617, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as

#### emilyalsentzer/Bio_ClinicalBERT (tok)

In [54]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "emilyalsentzer/Bio_ClinicalBERT" \
--batch_size 100 \
--average_token_embeddings

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: emilyalsentzer/Bio_ClinicalBERT
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: True
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=132,412155249657523952).pkl
len(self.cache["hashes"]) = 144285
self.cache["embeddings"].shape = (144285, 768)
100%|████████████████████████████████████| 8617/8617 [00:00<00:00, 32540.98it/s]
Embeddings shape: (8617, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as q

#### CheXbert

In [55]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "CheXbert" \
--device "cuda"

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: CheXbert
   device: cuda
   batch_size: 32
   num_workers: 4
   model_checkpoint_folder_path: None
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
100%|███████████████████████████████████████| 8617/8617 [01:34<00:00, 91.56it/s]
Embeddings shape: (8617, 768)
Normalizing embeddings (for cosine similarity)
Evaluating embeddings on ranking task with each sentence as query
100%|██████████████████████████████████████| 8617/8617 [01:00<00:00, 143.33it/s]
mean_AUC: 0.9327
mean_relevant: 63.7905
count: 7742 / 8617 (89.85%)


#### CXR Fact Encoder (T)

In [56]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230927_194923_MIMIC-CXR(triplets-only)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230927_194923_MIMIC-CXR(triplets-only)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
checkpoint_names = ['checkpoint_40_ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9355.pt']
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=359,388447562913600091).pkl
len(self.cache["hashes"]) = 1

#### CXR Fact Encoder (T -> T+EC)

In [57]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231003_001641_MIMIC-CXR(triplets+entcont)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231003_001641_MIMIC-CXR(triplets+entcont)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
checkpoint_names = ['checkpoint_1_encc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9444.pt']
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=366,4246634492375333681).pkl
len(self.cache["hashe

#### CXR Fact Encoder (T -> T+C)

In [58]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_013825_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_013825_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
checkpoint_names = ['checkpoint_64_cacc+chf1+chf1+cscc+hscc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9474.pt']
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=387,2492157991032224976).pkl


In [59]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_155915_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_155915_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
checkpoint_names = ['checkpoint_29_cacc+chf1+chf1+cscc+hscc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9495.pt']
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=387,1308360022390228203).pkl


#### CXR Fact Encoder (T -> T+EC+NLI)

In [60]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_220207_MIMIC-CXR(triplets+entcon+nli)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_220207_MIMIC-CXR(triplets+entcon+nli)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
checkpoint_names = ['checkpoint_52_encc+nlcc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9002.pt']
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=375,1493189425621868816).pkl
len(self.cac

#### CXR Fact Encoder (T -> T+R)

In [61]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_110825_MIMIC-CXR(triplets+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_110825_MIMIC-CXR(triplets+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
checkpoint_names = ['checkpoint_93_spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9415.pt']
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=368,1866338409828607810).pkl
len(self.cache["has

#### CXR Fact Encoder (T -> T+C+EC+NLI+R)


In [62]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_152825_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_152825_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
checkpoint_names = ['checkpoint_26_cacc+chf1+chf1+cscc+encc+hscc+nlcc+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9386.pt']
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cac

In [63]:
!python ../eval_fact_embedding_on_ranking.py \
--evaluation_mode "iuxray_with_automatic_labelers" \
--model_name "microsoft/BiomedVLP-CXR-BERT-specialized" \
--model_checkpoint_folder_path \
"/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_174626_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/" \
--batch_size 100

script's arguments:
   evaluation_mode: iuxray_with_automatic_labelers
   model_name: microsoft/BiomedVLP-CXR-BERT-specialized
   device: GPU
   batch_size: 100
   num_workers: 4
   model_checkpoint_folder_path: /mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_174626_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)/
   distance_metric: cosine
   average_token_embeddings: False
Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(thr1=0.4,thr2=0.2).pkl
len(sentences): 8617
len(relevant_sentences): 8617
checkpoint_names = ['checkpoint_50_cacc+chf1+chf1+cscc+encc+hscc+nlcc+spss+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9373.pt']
  0%|                                                  | 0/8617 [00:00<?, ?it/s]Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cac

In [5]:
from medvqa.eval_fact_embedding_on_ranking import SentenceRanker

sr = SentenceRanker('iuxray_with_automatic_labelers')

Loading iuxray sentences and relevant sentences from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/iuxray_sentences_and_relevant(rt=0.3).pkl


In [6]:
query_idx = 4006
print(sr.sentences[query_idx])
print('num_relevant:', len(sr.relevant_sentences[query_idx]))
print()
for i, idx in enumerate(sr.relevant_sentences[query_idx]):
    if i == 30: break
    print(sr.sentences[idx])

No pleural effusion or pneumothorax is identified.
num_relevant: 282

Negative for pneumothorax, pleural effusion or focal airspace consolidation.
No evidence of pulmonary consolidation, large pleural effusion/pneumothorax.
No definitive pneumothorax or pleural effusion.
No consolidation, pleural effusion or pneumothorax.
No definite pleural effusion seen, no pneumothorax.
No focal air space consolidation, pneumothorax, pleural effusion, or pulmonary edema.
There are no XXXX of a pleural effusion.
Previously seen left pleural effusion has resolved.
There is no acute pulmonary consolidation, pleural effusion or pneumothorax.
No definite pleural effusion.
There is no focal airspace disease, pneumothorax, or large pleural effusion.
There is no focal airspace opacity, large pleural effusion, or pneumothorax.
There is no pneumothorax, pleural effusion, or focal airspace consolidation.
Specifically, no evidence of focal consolidation, pneumothorax, or pleural effusion..
The lungs are clear w

In [7]:
output = sr.rank_sentences(
    query_idx,
    model_name="microsoft/BiomedVLP-CXR-BERT-specialized",
#     checkpoint_folder_path="/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_220207_MIMIC-CXR(triplets+entcon+nli)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)",
#     checkpoint_folder_path="/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20230928_013825_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)",
#     checkpoint_folder_path="/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231002_152825_MIMIC-CXR(triplets+classif+entcont+nli+radgraph)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)",
    checkpoint_folder_path="/mnt/data/pamessina/workspaces/medvqa-workspace/models/fact_embedding/20231004_155915_MIMIC-CXR(triplets+classif)_FactEncoder(microsoft-BiomedVLP-CXR-BERT-specialized)",
    top_k=30,
)
print(sr.sentences[query_idx])
print('num_relevant:', len(sr.relevant_sentences[query_idx]))
print('-'*100)
print()
for i in range(len(output[0])):
    print(output[0][i])
    print(output[1][i])
    print()

checkpoint_names = ['checkpoint_29_cacc+chf1+chf1+cscc+hscc+ta0)+ta1)+ta2)+ta0)+ta1)+ta2)+ta3)+ta4)+ta5)+ta6)+ta7)=0.9495.pt']


100%|███████████████████████████████████████████████████████████████████████████| 8617/8617 [00:00<00:00, 384255.65it/s]

Loading cached text embeddings from /mnt/workspace/pamessina/medvqa-workspace/cache/text_embeddings_cache(hash=387,1308360022390228203).pkl
len(self.cache["hashes"]) = 9782
self.cache["embeddings"].shape = (9782, 128)
No pleural effusion or pneumothorax is identified.
num_relevant: 282
----------------------------------------------------------------------------------------------------

No pleural effusion or pneumothorax is identified.
0

No pleural effusion or pneumothorax is noted.
1

No pleural effusion or pneumothorax is seen.
1

No pleural effusion or pneumothorax are seen.
1

No pneumothorax or pleural effusion is seen.
1

No pneumothorax or pleural effusion is noted.
1

No definitive pneumothorax or pleural effusion.
1

No radiographic evidence of pleural effusion or pneumothorax.
1

No pneumothorax or pleural effusion demonstrated.
1

No appreciable pleural effusion or pneumothorax.
1

No large pleural effusion or pneumothorax.
1

No definite pleural effusion or pneumothorax.
1